In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
import math
import dask.dataframe as dd
import os
pd.options.mode.chained_assignment = None  # default='warn'
from matplotlib import pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
def fillSeries(row):
    new_row = []
    size = len(row[1])
    if size < 13:
        for idx in range(len(row)):
            if idx == 0: new_row.append(row[idx])
            else: new_row.append([row[idx][0] for cnt in range(13 - size)] + row[idx])
    else:
        new_row = row
    return new_row

In [2]:
df = pd.read_csv("./train_data/train_data.csv")

In [2]:
def preprocess(data_path, label_path=False):
    dataset = pd.read_csv(data_path)
    if label_path: labels = pd.read_csv(label_path)


    categorical_ = dataset[['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']]

    categorical_['D_63'] = categorical_['D_63'].map({"CR": 1, "CO": 2, "CL": 3, "XZ": 4, "XM": 5, "XL": 6})
    categorical_['D_64'] = categorical_['D_64'].map({"O": 1, "R": 2, "nan": pd.NA, "U": 3, "-1": 4})

    categorical_ = categorical_.fillna(0)


    categorical_.astype('int16')


    dataset_ = dataset.drop(['customer_ID', 'B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68'], axis=1)


    timeint = pd.to_datetime(dataset['S_2']).astype(int)
    timeint = (timeint-timeint.min())/(timeint.max()-timeint.min())
    dataset_['S_2'] = timeint

    dataset_ = dataset_.fillna(0)

    dataset_.astype('float16')

    dataset_final = pd.concat([dataset[['customer_ID']], dataset_, categorical_], axis=1)

    dataset_final = dataset_final.groupby(["customer_ID"]).agg(list)
    if label_path: dataset_final = dataset_final.join(labels.set_index('customer_ID'))
    dataset_final = dataset_final.reset_index()
    return dataset_final

In [ ]:
def split_file(file_path, out_path, customers_per_file):
    # split the training dataset into smaller chunks
    customers_per_file = customers_per_file + 1
    ids = {}
    cnt = 0
    ids_file = open('{}/{}.csv'.format(out_path, "ids"), "w")
    ids_file.write("id,chunk\n")
    save = False
    with open(file_path) as bigfile:
        for lineno, line in enumerate(bigfile):
            if lineno == 0: continue    # skip header
            customer_ID = line.split(",")[0]    # get customer ID of current line

            sum_ids = len(ids.keys())

            # add customer to list of read customers
            if customer_ID in ids: ids[customer_ID].append(line)
            else:
                if sum_ids != (customers_per_file - 1): ids[customer_ID] = [line]
                else: save = True


            # write data of all read customers to a new file
            if save:
                data_file = open(f"{out_path}/data{cnt}.csv", "w")
                data_file.write("customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145\n")
                
                for key in ids:
                    data = ids[key]
                    # copy first element of series until series length is 13
                    if len(data) != 13: data = [data[0] for cnt in range(13 - len(data) )] + data 
                    ids_file.write(key + "," + str(cnt) + "\n")
                    for elem in data: data_file.write(elem)

                data_file.close()
                ids = {}
                ids[customer_ID] = [line]
                save = False
                cnt += 1

    data_file = open(f"{out_path}/data{cnt}.csv", "w")
    data_file.write("customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,D_44,B_4,D_45,B_5,R_2,D_46,D_47,D_48,D_49,B_6,B_7,B_8,D_50,D_51,B_9,R_3,D_52,P_3,B_10,D_53,S_5,B_11,S_6,D_54,R_4,S_7,B_12,S_8,D_55,D_56,B_13,R_5,D_58,S_9,B_14,D_59,D_60,D_61,B_15,S_11,D_62,D_63,D_64,D_65,B_16,B_17,B_18,B_19,D_66,B_20,D_68,S_12,R_6,S_13,B_21,D_69,B_22,D_70,D_71,D_72,S_15,B_23,D_73,P_4,D_74,D_75,D_76,B_24,R_7,D_77,B_25,B_26,D_78,D_79,R_8,R_9,S_16,D_80,R_10,R_11,B_27,D_81,D_82,S_17,R_12,B_28,R_13,D_83,R_14,R_15,D_84,R_16,B_29,B_30,S_18,D_86,D_87,R_17,R_18,D_88,B_31,S_19,R_19,B_32,S_20,R_20,R_21,B_33,D_89,R_22,R_23,D_91,D_92,D_93,D_94,R_24,R_25,D_96,S_22,S_23,S_24,S_25,S_26,D_102,D_103,D_104,D_105,D_106,D_107,B_36,B_37,R_26,R_27,B_38,D_108,D_109,D_110,D_111,B_39,D_112,B_40,S_27,D_113,D_114,D_115,D_116,D_117,D_118,D_119,D_120,D_121,D_122,D_123,D_124,D_125,D_126,D_127,D_128,D_129,B_41,B_42,D_130,D_131,D_132,D_133,R_28,D_134,D_135,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145\n")
                
    for key in ids:
        data = ids[key]
        # copy first element of series until series length is 13
        if len(data) != 13: data = [data[0] for cnt in range(13 - len(data) )] + data
        ids_file.write(key + "," + str(cnt) + "\n")
        for elem in data: data_file.write(elem)

    data_file.close()
    ids = {}
    cnt += 1

    ids_file.close()

In [ ]:
#split_file("train_data/train_data.csv", "train_data/train_data_chunks/", 10000)
#split_file("test_data/test_data.csv", "test_data/test_data_chunks/", 10000)

In [3]:
def preprocess_chunks(folder_path, label_path=False):    
    dfs = []
    for file in sorted(os.listdir(folder_path)):
        if file == "ids.csv": continue
        print(file)
        df = preprocess(os.path.join(folder_path, file), label_path)
        dfs.append(df)

    return pd.concat(dfs, axis=0)

In [4]:
#res = preprocess_chunks("train_data/train_data_chunks/", "train_data/train_labels.csv")
res = preprocess_chunks("test_data/test_data_chunks/")

data0.csv
data1.csv
data10.csv
data11.csv
data12.csv
data13.csv
data14.csv
data15.csv
data16.csv
data17.csv
data18.csv
data19.csv
data2.csv
data20.csv
data21.csv
data22.csv
data23.csv
data24.csv
data25.csv
data26.csv
data27.csv
data28.csv
data29.csv
data3.csv
data30.csv
data31.csv
data32.csv
data33.csv
data34.csv
data35.csv
data36.csv
data37.csv
data38.csv
data39.csv
data4.csv
data40.csv
data41.csv
data42.csv
data43.csv
data44.csv
data45.csv
data46.csv
data47.csv
data48.csv
data49.csv
data5.csv
data50.csv
data51.csv
data52.csv
data53.csv
data54.csv
data55.csv
data56.csv
data57.csv
data58.csv
data59.csv
data6.csv
data60.csv
data61.csv
data62.csv
data63.csv
data64.csv
data65.csv
data66.csv
data67.csv
data68.csv
data69.csv
data7.csv
data70.csv
data71.csv
data72.csv
data73.csv
data74.csv
data75.csv
data76.csv
data77.csv
data78.csv
data79.csv
data8.csv
data80.csv
data81.csv
data82.csv
data83.csv
data84.csv
data85.csv
data86.csv
data87.csv
data88.csv
data89.csv
data9.csv
data90.csv
data91.cs

In [5]:
#res.to_csv("./transformed_dataset.csv", chunksize=100)
res.to_csv("./transformed_test_dataset", chunksize=100)